# TMDB cleaning and creating movie embeddings

In [ ]:
# Data manipulation packages
import pandas as pd
import numpy as np

# data viz packages
import matplotlib.pyplot as plt
import seaborn as sns

## Get TMDB dataset

In [3]:
# Load in TMDB dataset
df_full = pd.read_csv(
    'data/tmdb_movie_dataset.csv',
    usecols = [
        'id', 
        'title', 'overview', 'tagline',                                     # useful for semantic similarity
        'genres', 'production_companies', 'keywords', 'release_date',       # useful features for semantic similarity, keyword matching and logical filtering
        'vote_average', 'vote_count', 'popularity',                         # useful or logical ranking and filtering (popular movies are more likely to be searched)
        'status', 'adult'                                                   # useful for filtering the dataset to get only non adult movies
    ],
    index_col = 'id'
)

df_full.head()

,title,vote_average,vote_count,status,release_date,adult,overview,popularity,tagline,genres,production_companies,keywords
id,,,,,,,,,,,,
27205,Inception,8.364,34495,Released,2010-07-15,False,"Cobb, a skilled thief who commits corporate es...",83.952,Your mind is the scene of the crime.,"Action, Science Fiction, Adventure","Legendary Pictures, Syncopy, Warner Bros. Pict...","rescue, mission, dream, airplane, paris, franc..."
157336,Interstellar,8.417,32571,Released,2014-11-05,False,The adventures of a group of explorers who mak...,140.241,Mankind was born on Earth. It was never meant ...,"Adventure, Drama, Science Fiction","Legendary Pictures, Syncopy, Lynda Obst Produc...","rescue, future, spacecraft, race against time,..."
155,The Dark Knight,8.512,30619,Released,2008-07-16,False,Batman raises the stakes in his war on crime. ...,130.643,Welcome to a world without rules.,"Drama, Action, Crime, Thriller","DC Comics, Legendary Pictures, Syncopy, Isobel...","joker, sadism, chaos, secret identity, crime f..."
19995,Avatar,7.573,29815,Released,2009-12-15,False,"In the 22nd century, a paraplegic Marine is di...",79.932,Enter the world of Pandora.,"Action, Adventure, Fantasy, Science Fiction","Dune Entertainment, Lightstorm Entertainment, ...","future, society, culture clash, space travel, ..."
24428,The Avengers,7.710,29166,Released,2012-04-25,False,When an unexpected enemy emerges and threatens...,98.082,Some assembly required.,"Science Fiction, Action, Adventure",Marvel Studios,"new york city, superhero, shield, based on com..."


In [4]:
# Select only released movies movies that are not adult films
df = df_full.loc[df_full['status'] == 'Released', df_full.columns.drop('status')]
df = df.loc[~(df['adult']), df.columns.drop('adult')]
df.head()

,title,vote_average,vote_count,release_date,overview,popularity,tagline,genres,production_companies,keywords
id,,,,,,,,,,
27205,Inception,8.364,34495,2010-07-15,"Cobb, a skilled thief who commits corporate es...",83.952,Your mind is the scene of the crime.,"Action, Science Fiction, Adventure","Legendary Pictures, Syncopy, Warner Bros. Pict...","rescue, mission, dream, airplane, paris, franc..."
157336,Interstellar,8.417,32571,2014-11-05,The adventures of a group of explorers who mak...,140.241,Mankind was born on Earth. It was never meant ...,"Adventure, Drama, Science Fiction","Legendary Pictures, Syncopy, Lynda Obst Produc...","rescue, future, spacecraft, race against time,..."
155,The Dark Knight,8.512,30619,2008-07-16,Batman raises the stakes in his war on crime. ...,130.643,Welcome to a world without rules.,"Drama, Action, Crime, Thriller","DC Comics, Legendary Pictures, Syncopy, Isobel...","joker, sadism, chaos, secret identity, crime f..."
19995,Avatar,7.573,29815,2009-12-15,"In the 22nd century, a paraplegic Marine is di...",79.932,Enter the world of Pandora.,"Action, Adventure, Fantasy, Science Fiction","Dune Entertainment, Lightstorm Entertainment, ...","future, society, culture clash, space travel, ..."
24428,The Avengers,7.710,29166,2012-04-25,When an unexpected enemy emerges and threatens...,98.082,Some assembly required.,"Science Fiction, Action, Adventure",Marvel Studios,"new york city, superhero, shield, based on com..."


In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1093220 entries, 27205 to 1506807
Data columns (total 10 columns):
 #   Column                Non-Null Count    Dtype  
---  ------                --------------    -----  
 0   title                 1093210 non-null  object 
 1   vote_average          1093220 non-null  float64
 2   vote_count            1093220 non-null  int64  
 3   release_date          892175 non-null   object 
 4   overview              859020 non-null   object 
 5   popularity            1093220 non-null  float64
 6   tagline               153742 non-null   object 
 7   genres                686387 non-null   object 
 8   production_companies  435145 non-null   object 
 9   keywords              269137 non-null   object 
dtypes: float64(2), int64(1), object(7)
memory usage: 91.7+ MB


## Clean dataset

In [6]:
def empty_vals(df):
    '''function to show both number of nulls and the percentage of nulls in the whole column'''
    total_cnt = len(df)                                                 # Total entries in the dataframe
    empty_strs = (df == '').sum()                                       # Number of empty strings in each column
    empty_lists = df.map(
        lambda x: isinstance(x, list) and len(x) == 0                   # Number of empty lists in each column
    ).sum()
    empty_lists_str = (df == '[]').sum()                                # Number of empty lists strings in each column
    nulls = (df.isnull()).sum()                                         # Number of nulls in each column
    totals = empty_strs + empty_lists + empty_lists_str + nulls         # Total number of empty/null values
    
    empty_vals = pd.DataFrame({         # Put all the data into a single dataframe
        '\'\'': empty_strs,
        '[]': empty_lists,
        '\'[]\'': empty_lists_str,
        'null': nulls,
        'total':  totals
    })
    empty_vals['percent'] = round((empty_vals['total']/total_cnt)*100, 3)       # Round how many empty/null values there are, as %, of the df
    
    return empty_vals.sort_values('percent', ascending=False)

empty_vals(df)

,'',[],'[]',null,total,percent
tagline,0,0,0,939478,939478,85.937
keywords,0,0,0,824083,824083,75.381
production_companies,0,0,0,658075,658075,60.196
genres,0,0,0,406833,406833,37.214
overview,0,0,0,234200,234200,21.423
release_date,0,0,0,201045,201045,18.390
title,0,0,1,10,11,0.001
vote_average,0,0,0,0,0,0.000
vote_count,0,0,0,0,0,0.000
popularity,0,0,0,0,0,0.000


In [7]:
df_cleaned = df.loc[
    (df["title"].notnull()) & 
    (df["release_date"].notnull()) & 
    (df["overview"].notnull())
]
empty_vals(df_cleaned)

,'',[],'[]',null,total,percent
tagline,0,0,0,607194,607194,82.372
keywords,0,0,0,508940,508940,69.043
production_companies,0,0,0,376505,376505,51.077
genres,0,0,0,191994,191994,26.046
title,0,0,1,0,1,0.000
vote_average,0,0,0,0,0,0.000
vote_count,0,0,0,0,0,0.000
release_date,0,0,0,0,0,0.000
overview,0,0,0,0,0,0.000
popularity,0,0,0,0,0,0.000


In [8]:
df_cleaned = df_cleaned.fillna("NA")
empty_vals(df_cleaned)

,'',[],'[]',null,total,percent
title,0,0,1,0,1,0.0
vote_average,0,0,0,0,0,0.0
vote_count,0,0,0,0,0,0.0
release_date,0,0,0,0,0,0.0
overview,0,0,0,0,0,0.0
popularity,0,0,0,0,0,0.0
tagline,0,0,0,0,0,0.0
genres,0,0,0,0,0,0.0
production_companies,0,0,0,0,0,0.0
keywords,0,0,0,0,0,0.0


In [9]:
df_cleaned.info()

<class 'pandas.core.frame.DataFrame'>
Index: 737133 entries, 27205 to 1506807
Data columns (total 10 columns):
 #   Column                Non-Null Count   Dtype  
---  ------                --------------   -----  
 0   title                 737133 non-null  object 
 1   vote_average          737133 non-null  float64
 2   vote_count            737133 non-null  int64  
 3   release_date          737133 non-null  object 
 4   overview              737133 non-null  object 
 5   popularity            737133 non-null  float64
 6   tagline               737133 non-null  object 
 7   genres                737133 non-null  object 
 8   production_companies  737133 non-null  object 
 9   keywords              737133 non-null  object 
dtypes: float64(2), int64(1), object(7)
memory usage: 61.9+ MB


## Feature engineering

In [12]:
# Create 'movie_details' column that will be vectorised
df_cleaned['movie_details'] = "title: " + df_cleaned["title"]
df_cleaned['movie_details'] += " release_date: " + df_cleaned["release_date"] 
df_cleaned['movie_details'] += " genres: " + df_cleaned["genres"]
df_cleaned['movie_details'] += " tagline: " + df_cleaned["tagline"] 
df_cleaned['movie_details'] += " keywords: " + df_cleaned["keywords"]  
df_cleaned['movie_details'] += " overview: " + df_cleaned["overview"] 
df_cleaned['movie_details'].head()

id
27205     title: Inception release_date: 2010-07-15 genr...
157336    title: Interstellar release_date: 2014-11-05 g...
155       title: The Dark Knight release_date: 2008-07-1...
19995     title: Avatar release_date: 2009-12-15 genres:...
24428     title: The Avengers release_date: 2012-04-25 g...
Name: movie_details, dtype: object

In [11]:
# Null check
df_cleaned["movie_details"].isnull().sum()

0

## Save dataset

In [ ]:
df_cleaned.to_csv('data/processed_dataset.csv')

: 